In [ ]:

from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [ ]:
class T(Module):
    def __init__(self): self.x = torch.ones(3)

In [ ]:
L(T().parameters())

(#0) []

In [ ]:
class T(Module):
    def __init__(self): self.x = nn.Parameter(torch.ones(3))

In [ ]:
L(T().parameters())

(#1) [Parameter containing:
tensor([1., 1., 1.], requires_grad=True)]

In [ ]:
class T(Module):
    def __init__(self): self.x = nn.Linear(1,3,bias=False)

In [ ]:
L(T().parameters())

(#1) [Parameter containing:
tensor([[-0.8660],
        [-0.4537],
        [ 0.8831]], requires_grad=True)]

In [ ]:
t =T()

In [ ]:
t.x.weight

Parameter containing:
tensor([[-0.7270],
        [ 0.4527],
        [-0.7421]], requires_grad=True)

In [ ]:
emb = nn.Embedding(100,20)

In [ ]:
emb.weight[10]

tensor([-1.1954,  0.9443,  0.3535,  0.8715,  1.0091, -1.8764,  1.4262, -0.3604,
        -0.0248, -1.0068,  0.0698, -2.1337, -0.9069, -0.7962,  2.2426, -0.9588,
        -1.5924, -0.6236, -1.2655,  0.7000], grad_fn=<SelectBackward0>)

In [ ]:
emb.weight[[1,2,33]]

tensor([[-3.4270e-01,  6.5162e-01,  1.0454e+00, -1.1925e+00,  4.5353e-02,
          1.9328e-01,  9.1166e-01,  1.2053e+00, -1.5770e-01,  5.4330e-01,
          1.1138e+00, -2.8420e-01,  2.2472e+00, -4.3706e-01,  1.0094e+00,
         -3.0744e+00,  2.8832e-02,  6.7263e-01,  3.4886e-01, -7.1864e-01],
        [-1.5175e-01,  9.4877e-01,  5.1240e-01, -1.0960e+00,  3.2397e-01,
          6.4362e-04, -1.2044e+00,  1.3355e+00,  1.1899e+00,  1.8298e-01,
          1.0810e-01, -1.1111e+00, -6.3136e-01, -3.9756e-02, -4.6154e-01,
          6.4018e-01, -5.2047e-01,  8.9511e-02,  2.5918e-01, -5.5907e-01],
        [-4.3536e-01,  1.0073e+00, -2.8394e-01,  5.2880e-02,  1.9095e+00,
          5.9261e-01,  2.2083e-01, -8.1232e-01,  1.7705e-01,  2.1331e-01,
          2.2464e+00,  1.7635e+00,  1.1908e+00,  6.9740e-01,  6.1847e-02,
          6.4433e-01,  8.0468e-01, -1.9007e-01,  3.5147e-01, -7.9033e-01]],
       grad_fn=<IndexBackward0>)

In [ ]:
path = untar_data(URLs.ML_100k)
path.ls()

ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                  names=['user','movie','rating','timestamp'])
ratings.head()

movies = pd.read_csv(path/'u.item', delimiter='|', header=None, encoding='latin1', 
                    usecols=[0,1], names=['movie','title'])
movies.head()

ratings= ratings.merge(movies)
ratings

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)
...,...,...,...,...,...
99995,880,476,3,880175444,"First Wives Club, The (1996)"
99996,716,204,5,879795543,Back to the Future (1985)
99997,276,1090,1,874795795,Sliver (1993)
99998,13,225,2,882399156,101 Dalmatians (1996)


In [ ]:
ratings

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)
...,...,...,...,...,...
99995,880,476,3,880175444,"First Wives Club, The (1996)"
99996,716,204,5,879795543,Back to the Future (1985)
99997,276,1090,1,874795795,Sliver (1993)
99998,13,225,2,882399156,101 Dalmatians (1996)


In [ ]:
CollabDataLoaders.from_df??

Signature:
CollabDataLoaders.from_df(
    ratings,
    valid_pct=0.2,
    user_name=None,
    item_name=None,
    rating_name=None,
    seed=None,
    path='.',
    *,
    bs: 'int' = 64,
    val_bs: 'int' = None,
    shuffle: 'bool' = True,
    device=None,
)
Source:   
    @delegates(DataLoaders.from_dblock)
    @classmethod
    def from_df(cls, ratings, valid_pct=0.2, user_name=None, item_name=None, rating_name=None, seed=None, path='.', **kwargs):
        "Create a `DataLoaders` suitable for collaborative filtering from `ratings`."
        user_name   = ifnone(user_name,   ratings.columns[0])
        item_name   = ifnone(item_name,   ratings.columns[1])
        rating_name = ifnone(rating_name, ratings.columns[2])
        cat_names = [user_name,item_name]
        splits = RandomSplitter(valid_pct=valid_pct, seed=seed)(range_of(ratings))
        to = TabularCollab(ratings, [Categorify], cat_names, y_names=[rating_name], y_block=TransformBlock(), splits=splits)
        return to.data

In [ ]:
dls= CollabDataLoaders.from_df(ratings,item_name='title')

In [ ]:
dls.show()

,user,movie,rating
95125,543,14,4
54806,571,462,4
79221,792,121,4
41820,393,841,3
81969,897,479,4
48317,441,121,4
92843,901,117,4
33426,262,420,3
24707,417,764,3
705,22,80,4


In [ ]:
n_users= len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_movies


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[1], line 2
      1 n_users= len(dls.classes['user'])
----> 2 n_movies = len(dls.classes['title'])
      3 n_movies

KeyError: 'title'


KeyError: 'title'